In [1]:
# création du model
from keras.models import Sequential

VGG16 = Sequential()  # réseau de neurones vides

In [2]:
from keras.layers import Conv2D, MaxPooling2D

# 1ere couche de convolution
VGG16.add(Conv2D(64,(3,3), input_shape=(224,224,3),
                padding="same", activation='relu'))
# 2e couche de convolution
VGG16.add(Conv2D(64,(3,3), padding='same', 
          activation='relu'))

# 1ere couche de pooling
VGG16.add(MaxPooling2D(pool_size=(2,2),
                      strides=(2,2)))

In [3]:
# Couches de pooling
from keras.layers import Flatten, Dense

# conversion des matrices 3D en vecteur 1D
VGG16.add(Flatten())

In [4]:
# 1ere couche de fully-connected
VGG16.add(Dense(4096, activation='relu'))

In [5]:
# Derniere couche de fully-connected
VGG16.add(Dense(1000, activation = 'softmax'))

In [6]:
# Appel du modèle VGG16 de keras pré-entrainé sur ImagetNet

from keras.applications.vgg16 import VGG16

model = VGG16()

In [7]:
# Nous allons utiliser ce réseau pour classer une image
# dans une des 1000 catégories de ImageNet.
# D'abord on charge l'image cat.jpg puis on le prétraite 
# pour qu'elle respecte les caractéristques des images en entrée
# de VGG16

from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import preprocess_input

img = load_img('cat.jpg', target_size=(224,224)) # chargement
img = img_to_array(img)  # conversion tableau
img = img.reshape((1, img.shape[0], img.shape[1], 
                  img.shape[2]))

# prétraiter suivant les caractéristiques de VGG16
img = preprocess_input(img)

In [8]:
pred = model.predict(img)

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [9]:
# TOP 5 des 1000 probabilés d'appartenance au 1000 catégories
# d'imageNet

from keras.applications.vgg16 import decode_predictions
print("Top 5 :", decode_predictions(pred, top=5))

Top 5 : [[('n02119789', 'kit_fox', 0.3743217), ('n02119022', 'red_fox', 0.36262238), ('n02124075', 'Egyptian_cat', 0.08030553), ('n02123159', 'tiger_cat', 0.054384407), ('n02123045', 'tabby', 0.029743355)]]


In [10]:
# Si toutefois nous sommes confrontés à un nouveau probleme
# de classification où la nouvelle image n'a pas etait entrainée
# parmi les 1000 catégories d'imageNet, on fait du 
# TRANSFER LERANING. Donc dans ce cas, on doit remplacer les dernieres
# couches fully-connected qui permmettent de classifier l'image
# dans une des 1000 catégories d'imageNet par un classifieur
# plus adpaté à notre probleme.

from keras.applications.vgg16 import VGG16
from keras.layers import Dense
from keras import Model

# chargement de VGG16 pré-entrainé sur ImageNet sans les couches 
# couches fully-connected
model = VGG16(weights='imagenet', include_top = False,
             input_shape=(224,224,3))

In [11]:
# Ajout de la couche de fully-connected pour 10 classes
predictions = Dense(10, activation='softmax')(model.output)

In [13]:
# Définir le nouveau modèle
n_model = Model(inputs=model.input, outputs=predictions)

In [14]:
n_model

In [22]:
# compilation
from keras import optimizers
n_model.compile(loss='categorical_crossentropy',
               optimizer=optimizers(lr=0.0001,momentum=0.9,
                                      metrics=['accuracy']))

TypeError: 'module' object is not callable

In [23]:
# Entraitement
model_train = n_model(xtrain, ytrain, 
                     epochs=100, 
                     batch_size=10
                     )

NameError: name 'xtrain' is not defined

In [ ]:
# Avec la stratégie 1 : fine-tuning total, on entraine 
# toutes les couches
for layer in model.layers:
    layer.trainable = True

In [ ]:
# Avec la stratégie 2 : extraxctions de features, on entraine 
# le nouveau classifieur et non les autres couches
for layer in model.layers:
    layer.trainable = False

In [ ]:
# Avec la stratégie 3 : fine-tuning partiel, on entraine 
# le nouveau classifieur et les couches hautes et non les 
# couches basses
for layer in model.layers[:5]:
    layer.trainable = False